In [ ]:
from __future__ import print_function

import numpy as np
import random
import multiprocessing as mp
from absl import app
from absl import flags
import ast
from network import Network
from config import get_config
from solver import Solver
from purification import Purification
import time
import os
FLAGS = flags.FLAGS

In [ ]:
def main(_):
    config = get_config(FLAGS) or FLAGS
    for num_paths in range(int(config.min_num_of_paths),int(config.num_of_paths)+1):
        for edge_capacity_bound in config.edge_capacity_bounds:
            for topology in config.list_of_topologies:
                purification = Purification()
                network = Network(config,purification,topology,edge_capacity_bound,False)
                print("config.purification_schemes",config.purification_schemes)
                #network.set_edge_fidelity(edge_fidelity_range)
                # we get all the paths for all workloads
                network.num_of_paths = num_paths
                for gate_fidelity_value in config.two_qubit_gate_fidelity_set:
                    network.purification.two_qubit_gate_fidelity = gate_fidelity_value
                    for measurement_fidelity in config.measurement_fidelity_set:
                        network.purification.measurement_fidelity = measurement_fidelity
                        network.set_nodes_q_value()
                        for number_of_flows in network.number_of_flow_set:
                            network.number_of_flows = number_of_flows
#                           for alpha_value in [0.0001,0.001,0.05,0.04,0.03,0.02,0.01,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]:
                            for alpha_value in config.alpha_values:
                                network.alpha_value = alpha_value
                                for min_rate in [2,4,6,8]:
                                    network.min_flow_rate = min_rate
                                    for target_long_link in [30,40,10,20,50,60]:
                                    
                                        for repeater_placement_distance in [5,10,15,20]:
                                            network.target_long_link=target_long_link
                                            network.repeater_placement_distance = repeater_placement_distance
                                            if target_long_link>repeater_placement_distance:
                                                network.engineer_topology(network.topology_file,target_long_link,repeater_placement_distance)
                                                network.load_topology()
                                                network.update_link_rates(alpha_value)
                                                for scheme in config.schemes:
                                                    print("for scheme %s flow size %s "%(scheme,number_of_flows))
                                                    network.running_path_selection_scheme = scheme
                                                    network.set_flows_of_organizations()
                                                    print("workloads loaded!")
                                                    #network.purification.set_each_wk_k_fidelity_threshold()
                                                    network.get_path_info()
                                                    print("path info extracted!")
                                                    if scheme in ["EGR","EGRSquare","Hop"]:
                                                        network.evaluate_shortest_path_routing(config,scheme)
                                                    elif scheme =="Genetic":
                                                        network.evaluate_genetic_algorithm_for_path_selection(config)
                                                    elif scheme =="RL":
                                                        network.evaluate_rl_for_path_selection(config)
                                                    else:
                                                        print("not valid scheme (%s): set schemes from EGR, EGRSquare,Hop, Genetic, or RL keywords"%(scheme))

In [ ]:
if __name__ == '__main__':
    app.run(main)